In [1]:
import torch
from fft_conv_pytorch import fft_conv, FFTConv1d, FFTConv2d
import time
import matplotlib.pyplot as plt

In [2]:
myNet = torch.nn.Sequential(
    torch.nn.Conv2d(1,20,5),
    torch.nn.ReLU(),
    torch.nn.Conv2d(20,64,3),
    torch.nn.ReLU()
)

In [8]:
print(myNet)
fir_layer = list(myNet.children())[0]

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
)


In [19]:
myNet[0]

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))

In [ ]:
torch.nn.SyncBatchNorm.convert_sync_batchnorm()

In [26]:
def hard_convert_opt_conv2d(module, threshold):
    module_output = module
    for i, layer in enumerate(list(myNet.children())):
        if isinstance(layer, torch.nn.Conv2d):
            if layer.kernel_size[0] > threshold:
                module_output[i] = FFTConv2d(in_channels=layer.in_channels, 
                                             out_channels=layer.out_channels,
                                             kernel_size=layer.kernel_size, 
                                             stride=layer.stride,
                                             padding=layer.padding,
                                             padding_mode=layer.padding_mode,
                                             dilation=layer.dilation,
                                             groups=layer.groups,
                                             bias=True if layer.bias is not None else False)
    del module
    return module_output

In [34]:
myNet_opt = hard_convert_opt_conv2d(myNet, 4)
print(myNet_opt)

Sequential(
  (0): _FFTConv()
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
)


### More candidates for convolution and matmul